In [1]:
!pip install grakel-dev

     |████████████████████████████████| 1.5MB 2.9MB/s 


In [0]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.keras.layers import BatchNormalization
from grakel import datasets as datasets_
import numpy as np
import scipy.sparse as sp
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import sklearn

In [4]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [1]:
#Folder where the experiments will be saved
folder = '.'

In [0]:
class GIN:
    
    def frobenius_norm(self, tensor):
        square_tensor = tf.square(tensor)
        tensor_sum = tf.reduce_sum(square_tensor)
        frobenius_norm = tf.sqrt(tensor_sum)
        return frobenius_norm
    
    def __variable(self,shape,reg=True,name='weight'):
        v = tf.get_variable(name=name,shape=shape,dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer())
        if reg:
            self.reg_variables.append(v)
        return v
    
    def __bias(self,shape,reg=True):
        v = tf.get_variable(name="bias",shape=shape,dtype=tf.float32,initializer=tf.keras.initializers.Zeros())
        if reg:
            self.reg_variables.append(v)
        return v
    
    def __eps(self,reg=True,name='eps'):
        v = tf.get_variable(name=name,shape=[1],dtype=tf.float32,initializer=tf.constant_initializer(0.1))
        if reg:
            self.reg_variables.append(v)
        return v

    def __mlp(self,X,k):
        
        with tf.variable_scope("MLP"):
            out = X
        
            for i in range(self.n_mlp_layers):
                
                with tf.variable_scope("layer_{0}".format(i)):
            
                    if i == 0 and k == 0:
                        W = self.__variable([self.Fin,self.Fhid])
                    else:
                        W = self.__variable([self.Fhid,self.Fhid])
                
                    b = self.__bias([1,self.Fhid])

                    bn = BatchNormalization()
                    out = tf.nn.relu((bn(tf.matmul(out,W) + b)))

            bn = BatchNormalization()
            return bn(out)
        
    def __init__(
        self,
        n_layers,
        n_mlp_layers,
        Fin,
        Fhid,
        Fout,
        with_eps,
        neighbor_pooling,
        gamma,
        idx_gpu,
        ):
        
        if neighbor_pooling not in ['mean','max','sum']:
            raise ValueError('{0} is not a valid neighbor pooling.'.format(neighbor_pooling))
        
        self.n_layers = n_layers
        self.n_mlp_layers = n_mlp_layers
        
        self.Fin = Fin
        self.Fhid = Fhid
        self.Fout = Fout
        
        self.gamma=gamma

        self.reg_variables = []
        
        with tf.device(idx_gpu):

                self.X = tf.placeholder(shape=[None,self.Fin],dtype=tf.float32,name="input")
                self.Y = tf.placeholder(shape=[None,self.Fout],dtype=tf.float32,name="output")
                self.A = tf.sparse.placeholder(shape=[None,None],dtype=tf.float32,name="adjacencies")
                self.R = tf.sparse.placeholder(shape=[None,None], dtype=tf.float32,name="dimensions")
                self.keep_prob = tf.placeholder(tf.float32)
                self.lr = tf.placeholder(tf.float32)

                H = self.X

                ############################################################
                # Model Implementation
                ############################################################

                # Assumptions: 
                # 1 - There are no self-loops
                # 2 - Every node has at least one neighbor
                
                if neighbor_pooling in ['sum','mean']:
                    
                    Adim = tf.shape(self.A)[0]
                    eye = tf.sparse.eye(Adim,dtype=tf.float32)

                if neighbor_pooling in ['max','mean']:
                    
                    n_neighbors = tf.sparse.reduce_sum(self.A,axis=0)
                    
                    if neighbor_pooling == 'mean':
                        # We extend the dimension in order to automatically
                        # do broadcasting when performing Q = Q / n_neighbors
                        
                        n_neighbors = tf.expand_dims(n_neighbors,1)
                
                if neighbor_pooling == 'max': 

                    c = tf.cumsum(n_neighbors)
                    seq = tf.searchsorted(c, tf.range(c[-1]), side='right')

                    idx = tf.reshape(self.A.indices[:,1],[-1])
                
                S = 0

                for k in range(self.n_layers-1):

                    with tf.variable_scope("layer_{0}".format(k)):

                        if neighbor_pooling in ['sum','mean']:
                            
                            A = self.A                            

                            mult = 1

                            if with_eps:
                                eps = self.__eps()
                                mult = 1+eps

                            if neighbor_pooling == 'mean':
                                mult = mult * n_neighbors

                            Aeps = eye.__mul__(mult)

                            A = tf.sparse.add(Aeps,A)

                            Q = tf.sparse.matmul(A,H) # M x Fin
                            
                            if neighbor_pooling == 'mean':
                                
                                Q = tf.math.divide(Q,n_neighbors)
                            
                        elif neighbor_pooling == 'max':
                            
                            H_ = H
                            
                            if with_eps:
                                eps = self.__eps()
                                H_ = (1+eps) * H_
                            
                            ga = tf.gather(H,idx,axis=0)
                            
                            Q = H_ + tf.segment_max(ga,seq)

                        H = self.__mlp(Q,k) # M x Fhid

                        P = tf.sparse.matmul(self.R,H) # n_graphs x Fhid

                        W = self.__variable([self.Fhid,self.Fout])
                        b = self.__bias([1,self.Fout])

                        T = tf.matmul(P,W) + b  # n_graphs x Fout
                        T = tf.nn.dropout(T,  self.keep_prob)

                        S += T # n_graphs x Fout

                self.logits = S

                ############################################################

                #loss function definition

                with tf.variable_scope('loss'):
                    self.l2_reg = 0
                    for W in self.reg_variables:
                        self.l2_reg += tf.nn.l2_loss(W)

                    self.data_loss = tf.reduce_mean(
                                        tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits,
                                                                                   labels=self.Y))

                    self.loss = self.data_loss + self.gamma * self.l2_reg

                #solver definition
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
                self.opt_step = self.optimizer.minimize(self.loss)

                #predictions and accuracy extraction
                self.predictions = tf.argmax(tf.nn.softmax(self.logits), 1)
                self.accuracy = tf.contrib.metrics.accuracy(self.predictions, tf.argmax(self.Y, 1))

                #gradients computation
                self.trainable_variables = tf.trainable_variables()
                self.var_grad = tf.gradients(self.loss, tf.trainable_variables())

                self.norm_grad = self.frobenius_norm(tf.concat([tf.reshape(g, [-1]) for g in self.var_grad], 0))
                    
############################################################
# helper functions for data preprocessing
############################################################

def coo_to_sparse_tensor(X):
    indices = np.mat([X.row, X.col]).transpose()
    return tf.SparseTensorValue(indices, X.data, X.shape)
    
def create_matrices(graphs,features):
    
    M = 0
    n_graphs = len(graphs)

    R_row = np.array([])
    R_col = np.array([]) 

    k = 0
    for i,g in enumerate(graphs):
        dim = np.shape(g)[0]
        M += dim

        R_row = np.concatenate((R_row,np.full(dim,i)))
        R_col = np.concatenate((R_col,np.arange(k,k+dim)))
        k += dim


    A = sp.block_diag(graphs,format='coo')
    A = coo_to_sparse_tensor(A)

    R_data = np.ones(M)
    R = sp.coo_matrix((R_data, (R_row, R_col)), shape=(n_graphs, M))
    R = coo_to_sparse_tensor(R)
    
    X = np.vstack(features)
    
    return A,R,X

def feature_matrix(n_nodes,features):
    f = []
    for i in range(n_nodes):
        f.append(np.array(features[i]))
    f = np.vstack(f)
    return f

def ohe_inputs(feature_matrices):
    categories = []
    n_features = np.shape(feature_matrices[0])[1]
    for i in range(n_features):
        categories.append([])
    
    for f in feature_matrices:
        for i in range(n_features):
            unique = np.unique(f[:,i])
            categories[i] = np.unique(np.concatenate((categories[i],unique),axis=0))
    
    ohe = OneHotEncoder(categories=categories)
    
    new_feature_matrices = []
    for f in feature_matrices:
        new_feature_matrices.append(ohe.fit_transform(f).toarray()) 
        # if tf would have support for sparse * sparse matmul, then it would make sense to keep the input
        # as a sparse matrix
        
    return new_feature_matrices

############################################################
                        
def run(dataset_name,
        neighbor_pooling = 'sum',
        batch_size = 32,      
        with_eps = True,
        ohe_input = True,
        save_files = True,
        folder = None):
  
    ############################################################
    # Parameters
    ############################################################

    seed = None
    ran = np.random.RandomState(seed=seed)

    
    n_epochs = 350
    n_iter_per_epoch = 50
    gamma = 0.0005
    n_splits= 10
    n_layers = 5
    n_mlp_layers = 2
    hidden_dim = 64

    learning_rate = 0.01
    decay_lr_every_n_epochs = 50 
    decay_factor = 0.5

    gamma=5e-4 # weight regularization

    idx_gpu='/GPU:0'

    print_every_n_its = 500
    avg_of_last_n_its = 500

    ############################################################
    # Data pre-processing
    ############################################################

    data = datasets_.fetch_dataset(dataset_name,
                                  verbose=True,
                                  as_graphs=True,
                                  with_classes=True,
                                  prefer_attr_nodes=True,
                                  produce_labels_nodes=True)
    graphs, Y = data.data, np.array(data.target)

    n_classes = len(np.unique(Y))
    n_graphs = len(graphs)

    enc = OneHotEncoder()
    Yohe = enc.fit_transform(np.reshape(Y + np.abs(np.min(Y)),[n_graphs,1])).toarray()

    adjacencies = []
    features = []

    for i,g in enumerate(graphs):

        A = g.get_adjacency_matrix()
        n_nodes = np.shape(A)[0]

        f = g.get_labels(label_type='vertex',purpose='adjacency')
        features.append(feature_matrix(n_nodes,f))
        adjacencies.append(sp.coo_matrix(A))

    graphs = np.array(adjacencies)
    
    if ohe_input is True:
        features = ohe_inputs(features)
    
    features = np.array(features)

    n_features = np.shape(features[0])[1]

    print('Finished extracting data.')

    Fin = n_features
    Fhid = hidden_dim
    Fout = n_classes

    print("Dataset {0}. Samples {1}. Sample Features {2}. Number of Classes {3}."\
              .format(dataset_name,n_graphs,n_features,n_classes))

    def __learning_rate(lr,lr_decay,it_decay,it):
        return lr * (lr_decay**int(np.floor(it /it_decay)))

    n_total_iterations = n_epochs * n_iter_per_epoch        

    test_costs = []
    test_accs = []
    test_times = []

    split = 0
    skf = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state = ran)
    for train_idx, test_idx in skf.split(np.arange(n_graphs), Y):

        print("Split {0}. Train Samples {1}. Test Samples {2}.".format(split,len(train_idx),len(test_idx)))
        split+=1

        ############################################################
        # Training
        ############################################################
        
        with tf.Graph().as_default() as g: # it should create a new graph for every fold

            model = GIN(n_layers,
                        n_mlp_layers,
                        Fin,
                        Fhid,
                        Fout,
                        with_eps,
                        neighbor_pooling,
                        gamma,
                        idx_gpu)

            dataset = tf.data.Dataset.from_tensor_slices((train_idx))
            dataset = dataset.shuffle(buffer_size=len(train_idx),seed=seed)\
                                            .batch(batch_size=batch_size)\
                                            .repeat()
            iterator = dataset.make_initializable_iterator()
            next_batch = iterator.get_next()

            # session creation and initilization
            config = tf.ConfigProto(allow_soft_placement = True)
            config.gpu_options.allow_growth = True
            session = tf.Session(config=config)
            init = tf.global_variables_initializer()
            session.run(init)
            session.run(iterator.initializer)    

            train_grad_norms = []
            train_costs = []
            train_accs = []
            train_times = []

            print("IT \t NORM \t\t LOSS \t\t ACC \t\t TIME")

            for it in range(n_total_iterations):

                batch_idx = session.run(next_batch)
                graphs_batch = graphs[batch_idx]
                features_batch = features[batch_idx]

                A_batch, R_batch, X_batch = create_matrices(graphs_batch,features_batch)
                Y_batch = Yohe[batch_idx]

                lr = __learning_rate(lr=learning_rate,
                                     lr_decay=decay_factor,
                                     it_decay=decay_lr_every_n_epochs*n_iter_per_epoch,
                                     it=it)

                tic = time.time()
                train_dict = {model.Y: Y_batch,
                              model.X: X_batch,
                              model.R: R_batch,
                              model.A : A_batch,
                              model.keep_prob: 0.5,
                              model.lr:lr}

                _, train_loss, norm_grad, train_acc =  session.run(
                                            [model.opt_step, model.loss, model.norm_grad, model.accuracy], train_dict) 

                train_time = time.time() - tic   

                train_grad_norms.append(norm_grad)
                train_costs.append(train_loss)
                train_accs.append(train_acc)
                train_times.append(train_time)

                if it % print_every_n_its == 0 or it == n_total_iterations-1:                
                    print("{0} \t {1:.3f}+-{2:.3f} \t {3:.3f}+-{4:.3f} \t {5:.3f}+-{6:.3f} \t {7:.3f}+-{8:.3f}".format(
                        it,
                        np.mean(train_grad_norms[-avg_of_last_n_its:]),
                        np.std(train_grad_norms[-avg_of_last_n_its:]),
                        np.mean(train_costs[-avg_of_last_n_its:]),
                        np.std(train_costs[-avg_of_last_n_its:]),
                        np.mean(train_accs[-avg_of_last_n_its:]),
                        np.std(train_accs[-avg_of_last_n_its:]),
                        np.mean(train_times[-avg_of_last_n_its:]),
                        np.std(train_times[-avg_of_last_n_its:])))

            if save_files:              
              np.save(folder + '{0}_train_costs_{1}_{2}_{3}_{4}'.format(
                  dataset_name,split-1,neighbor_pooling,batch_size,with_eps), train_costs)
              np.save(folder + '{0}_train_accs_{1}_{2}_{3}_{4}'.format(
                  dataset_name,split-1,neighbor_pooling,batch_size,with_eps), train_accs)
                               
            ############################################################
            # Testing
            ############################################################

            graphs_test = graphs[test_idx]
            features_test = features[test_idx]

            A_test, R_test, X_test = create_matrices(graphs_test,features_test)
            Y_test = Yohe[test_idx]

            tic = time.time()
            test_dict = {model.Y: Y_test,
                          model.X: X_test,
                          model.R: R_test,
                          model.A : A_test,
                          model.keep_prob: 1}

            test_loss, test_acc =  session.run([model.loss,model.accuracy], test_dict) 
            test_time = time.time() - tic

            test_times.append(test_time)
            test_costs.append(test_loss)
            test_accs.append(test_acc)

            print("Split {0}. Test. Loss {1:.3f}. Acc {2:.3f}. Time {3:.3f}."\
                      .format(split-1,test_loss,test_acc,test_time))

    print("Summary. Test. Loss {0:.3f}+-{1:.3f}. Acc {2:.3f}+-{3:.3f}. Time {4:.3f}+-{5:.3f}"\
             .format(np.mean(test_costs),
                    np.std(test_costs),
                    np.mean(test_accs),
                    np.std(test_accs),
                    np.mean(test_times),
                    np.std(test_times)))
    
    if save_files:
      np.save(folder + '{0}_test_costs_{1}_{2}_{3}'.format(
          dataset_name,neighbor_pooling,batch_size,with_eps),test_costs)
      np.save(folder + '{0}_test_accs_{1}_{2}_{3}'.format(
          dataset_name,neighbor_pooling,batch_size,with_eps),test_accs)


In [0]:
dataset = 'MUTAG'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 32,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Finished extracting data.
Dataset MUTAG. Samples 188. Sample Features 7. Number of Classes 2.
Split 0. Train Samples 168. Test Samples 20.


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 1086.183+-0.000 	 62.344+-0.000 	 0.406+-0.000 	 1.423+-0.000
500 	 15.703+-21.560 	 0.988+-2.472 	 0.751+-0.108 	 0.020+-0.001
1000 	 6.083+-3.635 	 0.496+-0.198 	 0.816+-0.091 	 0.020+-0.001
1500 	 4.205+-2.692 	 0.451+-0.155 	 0.828+-0.081 	 0.020+-0.001
2000 	 3.441+-2.691 	 0.432+-0.160 	 0.833+-0.086 	 0.020+-0.001
2500 	 2.765+-1.790 	 0.401+-0.146 	 0.846+-0.077 	 0.020+-0.002
3000 	 2.230+-1.545 	 0.361+-0.131 	 0.869+-0.070 	 0.021+-0.002
3500 	 2.431+-1.818 	 0.359+-0.140 	 0.864+-0.069 	 0.022+-0.002
4000 	 2.373+-1.676 	 0.347+-0.132 	 0.869+-0.073 	 0.020+-0.001
4500 	 2.732+-2.121 	 0.334+-0.136 	 0.877+-0.072 	 0.020+-0.001
5000 	 2.541+-1.760 	 0.282+-0.138 	 0.897+-0.061 	 0.021+-0.001
5500 	 2.588+-2.095 	 0.230+-0.112 	 0.917+-0.059 	 0.020+-0.001
6000 	 3.295+-3.652 	 0.243+-0.147 	 0.912+-0.061 	 0.020+-0.001
6500 	 2.751+-2.135 	 0.191+-0.099 	 0.937+-0.053 	 0.021+-0.002
7000 	 3.895+-4.574 	 0.214+-0.149 	 0.927+-0.059 	 0.0

In [0]:
dataset = 'MUTAG'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 128,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Finished extracting data.
Dataset MUTAG. Samples 188. Sample Features 7. Number of Classes 2.
Split 0. Train Samples 168. Test Samples 20.


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 610.297+-0.000 	 25.162+-0.000 	 0.508+-0.000 	 1.584+-0.000
500 	 13.360+-27.547 	 0.867+-2.526 	 0.776+-0.090 	 0.025+-0.003
1000 	 5.482+-3.535 	 0.390+-0.091 	 0.865+-0.049 	 0.026+-0.002
1500 	 3.900+-2.999 	 0.311+-0.095 	 0.899+-0.041 	 0.024+-0.002
2000 	 3.511+-4.709 	 0.279+-0.161 	 0.910+-0.050 	 0.024+-0.002
2500 	 2.984+-2.564 	 0.218+-0.078 	 0.933+-0.034 	 0.024+-0.002
3000 	 2.734+-3.892 	 0.162+-0.119 	 0.960+-0.032 	 0.024+-0.002
3500 	 2.082+-3.139 	 0.126+-0.093 	 0.972+-0.028 	 0.026+-0.003
4000 	 1.397+-2.782 	 0.082+-0.048 	 0.987+-0.018 	 0.024+-0.002
4500 	 1.946+-4.501 	 0.099+-0.103 	 0.981+-0.026 	 0.024+-0.002
5000 	 2.131+-4.855 	 0.109+-0.123 	 0.977+-0.031 	 0.024+-0.002
5500 	 0.371+-0.507 	 0.051+-0.012 	 0.993+-0.011 	 0.024+-0.002
6000 	 1.436+-5.804 	 0.064+-0.078 	 0.990+-0.018 	 0.026+-0.003
6500 	 0.361+-0.440 	 0.045+-0.012 	 0.994+-0.009 	 0.024+-0.002
7000 	 0.321+-0.385 	 0.041+-0.011 	 0.994+-0.009 	 0.02

In [0]:
dataset = 'MUTAG'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 32,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Finished extracting data.
Dataset MUTAG. Samples 188. Sample Features 7. Number of Classes 2.
Split 0. Train Samples 168. Test Samples 20.


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 948.284+-0.000 	 50.347+-0.000 	 0.312+-0.000 	 1.462+-0.000
500 	 20.245+-30.542 	 1.235+-3.390 	 0.717+-0.115 	 0.018+-0.001
1000 	 8.385+-6.098 	 0.515+-0.205 	 0.794+-0.091 	 0.018+-0.001
1500 	 6.044+-4.884 	 0.434+-0.156 	 0.832+-0.083 	 0.018+-0.001
2000 	 5.240+-4.713 	 0.366+-0.140 	 0.858+-0.077 	 0.018+-0.001
2500 	 4.016+-4.538 	 0.329+-0.153 	 0.875+-0.076 	 0.018+-0.001
3000 	 2.546+-1.816 	 0.246+-0.099 	 0.908+-0.060 	 0.018+-0.002
3500 	 3.079+-3.766 	 0.247+-0.117 	 0.908+-0.062 	 0.019+-0.002
4000 	 2.869+-2.754 	 0.217+-0.105 	 0.923+-0.055 	 0.018+-0.001
4500 	 3.741+-4.337 	 0.307+-0.169 	 0.883+-0.079 	 0.018+-0.001
5000 	 3.424+-3.798 	 0.231+-0.116 	 0.914+-0.065 	 0.018+-0.001
5500 	 2.284+-2.514 	 0.154+-0.080 	 0.947+-0.047 	 0.018+-0.001
6000 	 6.217+-23.093 	 0.281+-0.462 	 0.903+-0.087 	 0.018+-0.001
6500 	 3.992+-2.703 	 0.241+-0.092 	 0.902+-0.063 	 0.018+-0.001
7000 	 3.643+-2.665 	 0.217+-0.083 	 0.917+-0.054 	 0.0

In [0]:
dataset = 'MUTAG'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size =128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Finished extracting data.
Dataset MUTAG. Samples 188. Sample Features 7. Number of Classes 2.
Split 0. Train Samples 168. Test Samples 20.


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 250.093+-0.000 	 20.794+-0.000 	 0.578+-0.000 	 1.552+-0.000
500 	 14.883+-68.380 	 1.207+-9.652 	 0.815+-0.083 	 0.022+-0.002
1000 	 6.597+-6.163 	 0.394+-0.205 	 0.869+-0.059 	 0.020+-0.002
1500 	 4.142+-2.316 	 0.309+-0.070 	 0.888+-0.037 	 0.020+-0.002
2000 	 4.687+-3.757 	 0.317+-0.105 	 0.888+-0.046 	 0.020+-0.002
2500 	 4.233+-3.641 	 0.274+-0.099 	 0.906+-0.042 	 0.020+-0.002
3000 	 3.344+-3.364 	 0.194+-0.069 	 0.939+-0.031 	 0.020+-0.002
3500 	 3.600+-3.244 	 0.182+-0.056 	 0.943+-0.027 	 0.022+-0.002
4000 	 4.298+-5.302 	 0.203+-0.120 	 0.934+-0.040 	 0.020+-0.001
4500 	 3.751+-4.050 	 0.182+-0.085 	 0.939+-0.036 	 0.020+-0.001
5000 	 4.375+-5.264 	 0.241+-0.143 	 0.914+-0.045 	 0.021+-0.002
5500 	 3.400+-3.077 	 0.141+-0.062 	 0.954+-0.035 	 0.020+-0.002
6000 	 4.749+-7.261 	 0.130+-0.170 	 0.966+-0.032 	 0.021+-0.002
6500 	 4.622+-8.426 	 0.129+-0.139 	 0.966+-0.043 	 0.022+-0.002
7000 	 4.196+-3.073 	 0.174+-0.054 	 0.939+-0.031 	 0.02

In [0]:
dataset = 'MUTAG'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 32,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Finished extracting data.
Dataset MUTAG. Samples 188. Sample Features 7. Number of Classes 2.
Split 0. Train Samples 168. Test Samples 20.


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 107.386+-0.000 	 4.875+-0.000 	 0.438+-0.000 	 1.654+-0.000
500 	 9.197+-10.759 	 0.824+-1.603 	 0.757+-0.107 	 0.022+-0.002
1000 	 3.149+-1.752 	 0.468+-0.157 	 0.824+-0.084 	 0.022+-0.002
1500 	 2.433+-1.464 	 0.439+-0.158 	 0.830+-0.083 	 0.022+-0.002
2000 	 2.257+-1.416 	 0.422+-0.150 	 0.840+-0.074 	 0.024+-0.002
2500 	 2.174+-1.310 	 0.421+-0.143 	 0.834+-0.077 	 0.022+-0.002
3000 	 1.812+-1.107 	 0.394+-0.149 	 0.849+-0.082 	 0.023+-0.002
3500 	 1.736+-1.108 	 0.388+-0.140 	 0.845+-0.075 	 0.022+-0.002
4000 	 1.748+-1.107 	 0.386+-0.134 	 0.850+-0.072 	 0.022+-0.002
4500 	 1.753+-1.073 	 0.379+-0.124 	 0.845+-0.075 	 0.022+-0.002
5000 	 1.754+-1.028 	 0.379+-0.128 	 0.847+-0.079 	 0.023+-0.002
5500 	 1.589+-0.876 	 0.364+-0.115 	 0.856+-0.072 	 0.022+-0.002
6000 	 1.581+-0.880 	 0.365+-0.123 	 0.856+-0.071 	 0.022+-0.002
6500 	 1.682+-1.136 	 0.368+-0.127 	 0.851+-0.071 	 0.022+-0.001
7000 	 1.624+-0.960 	 0.371+-0.129 	 0.855+-0.070 	 0.022+

In [0]:
dataset = 'MUTAG'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 128,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Initializing folder at /root/grakel_data
Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Finished extracting data.
Dataset MUTAG. Samples 188. Sample Features 7. Number of Classes 2.
Split 0. Train Samples 168. Test Samples 20.
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 158.584+-0.000 	 10.481+-0.000 	 0.352+-0.000 	 2.306+-0.000
500 	 7.348+-7.575 	 0.716+-1.177 	 0.760+-0.082 	 0.026+-0.002
1000 	 2.922+-1.728 	 0.463+-0.086 	 0.824+-0.046 	 0.027+-0.004
1500 	 1.805+-0.910 	 0.431+-0.071 	 0.837+-0.043 	 0.027+-0.004
2000 	 1.555+-0.836 	 0.420+-0.069 	 0.838+-0.039 	 0.027+-0.003
2500 	 1.321+-0.702 	 0.411+-0.070 	 0.839+-0.043 	 0.027+-0.004
3000 	 1.203+-0.665 	 0.397+-0.062 	 0.845+-0.039 	 0.026+-0.003
3500 	 1.266+-0.677 	 0.393+-0.067 	 0.849+-0.040 	 0.026+-0.002
4

In [0]:
dataset = 'MUTAG'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 32,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Finished extracting data.
Dataset MUTAG. Samples 188. Sample Features 7. Number of Classes 2.
Split 0. Train Samples 168. Test Samples 20.


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 75.782+-0.000 	 7.629+-0.000 	 0.500+-0.000 	 1.418+-0.000
500 	 10.912+-13.458 	 0.963+-1.695 	 0.728+-0.115 	 0.018+-0.002
1000 	 3.437+-2.098 	 0.487+-0.156 	 0.816+-0.082 	 0.018+-0.002
1500 	 2.620+-1.553 	 0.464+-0.147 	 0.824+-0.087 	 0.020+-0.002
2000 	 2.372+-1.450 	 0.457+-0.155 	 0.827+-0.083 	 0.018+-0.002
2500 	 2.129+-1.136 	 0.431+-0.125 	 0.835+-0.074 	 0.018+-0.002
3000 	 1.792+-1.101 	 0.414+-0.129 	 0.850+-0.070 	 0.018+-0.001
3500 	 1.843+-1.096 	 0.407+-0.132 	 0.850+-0.076 	 0.018+-0.002
4000 	 1.878+-1.201 	 0.413+-0.128 	 0.847+-0.076 	 0.019+-0.002
4500 	 1.911+-1.171 	 0.408+-0.139 	 0.846+-0.075 	 0.019+-0.002
5000 	 1.907+-1.047 	 0.407+-0.125 	 0.847+-0.076 	 0.018+-0.002
5500 	 1.771+-1.174 	 0.406+-0.135 	 0.851+-0.074 	 0.020+-0.002
6000 	 1.782+-1.046 	 0.394+-0.132 	 0.852+-0.081 	 0.018+-0.002
6500 	 1.800+-1.131 	 0.396+-0.131 	 0.851+-0.077 	 0.018+-0.002
7000 	 1.841+-1.189 	 0.403+-0.137 	 0.852+-0.072 	 0.018+

In [0]:
dataset = 'MUTAG'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Finished extracting data.
Dataset MUTAG. Samples 188. Sample Features 7. Number of Classes 2.
Split 0. Train Samples 168. Test Samples 20.


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 56.780+-0.000 	 5.637+-0.000 	 0.578+-0.000 	 1.397+-0.000
500 	 7.392+-7.664 	 0.703+-1.146 	 0.761+-0.079 	 0.021+-0.001
1000 	 2.451+-1.220 	 0.442+-0.077 	 0.829+-0.042 	 0.021+-0.001
1500 	 1.804+-0.916 	 0.411+-0.071 	 0.842+-0.040 	 0.021+-0.001
2000 	 1.696+-0.967 	 0.395+-0.072 	 0.847+-0.043 	 0.021+-0.001
2500 	 1.701+-0.933 	 0.377+-0.071 	 0.854+-0.039 	 0.021+-0.002
3000 	 1.545+-0.881 	 0.343+-0.061 	 0.867+-0.034 	 0.022+-0.002
3500 	 1.585+-0.969 	 0.340+-0.064 	 0.868+-0.037 	 0.021+-0.001
4000 	 1.622+-1.098 	 0.340+-0.065 	 0.866+-0.038 	 0.021+-0.001
4500 	 2.005+-1.351 	 0.317+-0.069 	 0.873+-0.039 	 0.021+-0.001
5000 	 1.905+-1.688 	 0.380+-0.076 	 0.850+-0.042 	 0.021+-0.001
5500 	 1.596+-0.923 	 0.360+-0.062 	 0.859+-0.038 	 0.023+-0.002
6000 	 1.933+-1.247 	 0.343+-0.063 	 0.863+-0.038 	 0.021+-0.002
6500 	 2.014+-1.366 	 0.337+-0.064 	 0.863+-0.040 	 0.021+-0.001
7000 	 1.892+-1.059 	 0.327+-0.063 	 0.867+-0.040 	 0.021+-0

In [0]:
dataset = 'MUTAG'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 32,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset MUTAG. Samples 188. Sample Features 7. Number of Classes 2.
Split 0. Train Samples 168. Test Samples 20.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 220.398+-0.000 	 12.389+-0.000 	 0.438+-0.000 	 1.735+-0.000
500 	 11.459+-13.556 	 0.992+-2.353 	 0.708+-0.121 	 0.021+-0.002
1000 	 3.806+-2.341 	 0.537+-0.158 	 0.789+-0.090 	 0.022+-0.002
1500 	 2.458+-1.367 	 0.474+-0.143 	 0.819+-0.075 	 0.021+-0.002
2000 	 2.241+-1.207 	 0.460+-0.139 	 0.822+-0.080 	 0.022+-0.002
2500 	 2.116+-1.175 	 0.445+-0.132 	 0.825+-0.073 	 0.022+-0.002
3000 	 2.054+-1.358 	 0.433+-0.154 	 0.826+-0.087 	 0.022+-0.002
3500 	 1.925+-1.208 	 0.420+-0.134 	 0.835+-0.070 	 0.023+-0.002
4000 	 1.906+-1.159 	 0.416+-0.133 	 0.833+-0.080 	 0.021+-0.002
4500 	 2.002+-1.269 	 0.413+-0.146 	 0.834+-0.083 	 0.021+-0.002
5000 	 2.117+-1.426 	 0.420+-0.151 	 0.827+-0.078 	 0.021+-0.002
5500 	 1.838+-1.112 	 0.394+-0.122 	 0.843+-0.069 	 0.022+-0.002
6000 	 1.855+-1.190 	 0.394+-0.143 	 0.839+-0.080 	 0.021+-0.002
6500 	 2.060+-1.594 	 0.400+-0.128 	 0.837+-0.074 	 0.022+-0.002
7000 	 1.895+-1.233 	 0.401+-0.138 	 0.835+-0.079 	 0.02

In [0]:
dataset = 'MUTAG'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 128,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset MUTAG. Samples 188. Sample Features 7. Number of Classes 2.
Split 0. Train Samples 168. Test Samples 20.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 64.511+-0.000 	 7.561+-0.000 	 0.594+-0.000 	 3.566+-0.000
500 	 8.283+-11.756 	 0.852+-1.836 	 0.722+-0.092 	 0.042+-0.007
1000 	 2.761+-1.611 	 0.466+-0.075 	 0.817+-0.046 	 0.040+-0.006
1500 	 1.825+-0.921 	 0.435+-0.071 	 0.829+-0.043 	 0.039+-0.006
2000 	 1.443+-0.762 	 0.420+-0.070 	 0.833+-0.042 	 0.042+-0.007
2500 	 1.373+-0.844 	 0.415+-0.068 	 0.832+-0.040 	 0.041+-0.007
3000 	 1.112+-0.606 	 0.413+-0.069 	 0.843+-0.040 	 0.040+-0.007
3500 	 1.102+-0.629 	 0.410+-0.068 	 0.844+-0.043 	 0.042+-0.007
4000 	 1.095+-0.618 	 0.409+-0.063 	 0.841+-0.041 	 0.040+-0.006
4500 	 1.072+-0.571 	 0.401+-0.058 	 0.844+-0.036 	 0.040+-0.006
5000 	 1.118+-0.619 	 0.402+-0.060 	 0.842+-0.041 	 0.043+-0.008
5500 	 1.020+-0.574 	 0.391+-0.061 	 0.846+-0.037 	 0.040+-0.007
6000 	 1.033+-0.568 	 0.390+-0.063 	 0.843+-0.039 	 0.040+-0.006
6500 	 1.054+-0.584 	 0.388+-0.064 	 0.844+-0.039 	 0.041+-0.007
7000 	 1.081+-0.612 	 0.386+-0.064 	 0.843+-0.038 	 0.040+-

In [0]:
dataset = 'MUTAG'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 32,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Finished extracting data.
Dataset MUTAG. Samples 188. Sample Features 7. Number of Classes 2.
Split 0. Train Samples 168. Test Samples 20.


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices t

IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 90.350+-0.000 	 11.745+-0.000 	 0.531+-0.000 	 1.487+-0.000
500 	 9.163+-9.980 	 0.835+-1.006 	 0.729+-0.120 	 0.020+-0.001
1000 	 2.918+-1.668 	 0.473+-0.142 	 0.816+-0.078 	 0.020+-0.002
1500 	 2.195+-1.371 	 0.440+-0.148 	 0.826+-0.080 	 0.021+-0.002
2000 	 2.057+-1.125 	 0.425+-0.132 	 0.828+-0.075 	 0.021+-0.002
2500 	 2.291+-1.352 	 0.431+-0.143 	 0.821+-0.081 	 0.024+-0.002
3000 	 1.810+-1.012 	 0.400+-0.121 	 0.831+-0.073 	 0.022+-0.002
3500 	 1.707+-0.910 	 0.390+-0.121 	 0.833+-0.078 	 0.022+-0.002
4000 	 1.903+-1.187 	 0.404+-0.130 	 0.829+-0.077 	 0.022+-0.002
4500 	 1.858+-1.081 	 0.398+-0.127 	 0.829+-0.079 	 0.022+-0.002
5000 	 1.939+-1.264 	 0.408+-0.129 	 0.826+-0.080 	 0.022+-0.002
5500 	 1.611+-0.857 	 0.376+-0.109 	 0.837+-0.073 	 0.024+-0.002
6000 	 1.742+-1.134 	 0.387+-0.130 	 0.834+-0.083 	 0.024+-0.003
6500 	 1.780+-1.106 	 0.383+-0.121 	 0.839+-0.072 	 0.022+-0.002
7000 	 1.792+-0.983 	 0.375+-0.115 	 0.843+-0.069 	 0.020+-

In [0]:
dataset = 'MUTAG'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Initializing folder at /root/grakel_data
Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Finished extracting data.
Dataset MUTAG. Samples 188. Sample Features 7. Number of Classes 2.
Split 0. Train Samples 168. Test Samples 20.


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 74.147+-0.000 	 5.627+-0.000 	 0.672+-0.000 	 2.166+-0.000
500 	 9.496+-22.380 	 0.924+-5.154 	 0.770+-0.081 	 0.022+-0.001
1000 	 3.404+-1.644 	 0.454+-0.081 	 0.832+-0.044 	 0.022+-0.001
1500 	 2.214+-1.030 	 0.417+-0.072 	 0.848+-0.040 	 0.022+-0.001
2000 	 1.798+-0.948 	 0.401+-0.076 	 0.847+-0.041 	 0.024+-0.002
2500 	 1.486+-0.890 	 0.394+-0.071 	 0.854+-0.042 	 0.024+-0.002
3000 	 1.426+-0.814 	 0.365+-0.069 	 0.861+-0.037 	 0.022+-0.002
3500 	 1.632+-0.970 	 0.345+-0.066 	 0.869+-0.033 	 0.022+-0.001
4000 	 1.649+-1.245 	 0.343+-0.072 	 0.867+-0.040 	 0.022+-0.001
4500 	 1.643+-1.067 	 0.338+-0.072 	 0.875+-0.039 	 0.024+-0.002
5000 	 1.544+-0.949 	 0.324+-0.065 	 0.876+-0.035 	 0.022+-0.002
5500 	 1.335+-0.814 	 0.305+-0.068 	 0.885+-0.035 	 0.022+-0.001
6000 	 1.462+-0.824 	 0.293+-0.061 	 0.887+-0.037 	 0.022+-0.001
6500 	 1.920+-1.172 	 0.273+-0.053 	 0.891+-0.034 	 0.022+-0.001
7000 	 2.296+-1.800 	 0.247+-0.062 	 0.898+-0.037 	 0.022+-

In [0]:
dataset = 'PROTEINS'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 32,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset PROTEINS. Samples 1113. Sample Features 61. Number of Classes 2.
Split 0. Train Samples 1001. Test Samples 112.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 1461.316+-0.000 	 92.721+-0.000 	 0.656+-0.000 	 1.680+-0.000
500 	 45.498+-182.029 	 3.970+-40.758 	 0.648+-0.106 	 0.024+-0.002
1000 	 14.402+-10.722 	 0.845+-0.361 	 0.726+-0.088 	 0.023+-0.002
1500 	 9.696+-6.252 	 0.640+-0.217 	 0.781+-0.081 	 0.023+-0.002
2000 	 7.574+-5.233 	 0.545+-0.196 	 0.821+-0.077 	 0.023+-0.002
2500 	 6.668+-5.642 	 0.499+-0.189 	 0.841+-0.077 	 0.023+-0.002
3000 	 5.301+-4.594 	 0.317+-0.108 	 0.922+-0.057 	 0.023+-0.002
3500 	 5.895+-5.989 	 0.387+-0.220 	 0.899+-0.072 	 0.025+-0.002
4000 	 4.861+-4.491 	 0.293+-0.122 	 0.936+-0.048 	 0.023+-0.002
4500 	 5.402+-5.517 	 0.343+-0.223 	 0.919+-0.072 	 0.023+-0.002
5000 	 4.908+-4.959 	 0.320+-0.170 	 0.930+-0.059 	 0.023+-0.002
5500 	 2.952+-5.341 	 0.194+-0.130 	 0.979+-0.031 	 0.023+-0.002
6000 	 2.272+-3.722 	 0.188+-0.127 

In [0]:
dataset = 'PROTEINS'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 128,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset PROTEINS. Samples 1113. Sample Features 61. Number of Classes 2.
Split 0. Train Samples 1001. Test Samples 112.
Instructions for updating:
Use tf.cast instead.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 764.929+-0.000 	 34.339+-0.000 	 0.625+-0.000 	 2.787+-0.000
500 	 49.989+-364.915 	 4.487+-58.377 	 0.688+-0.076 	 0.046+-0.008
1000 	 17.089+-12.867 	 0.662+-0.241 	 0.799+-0.058 	 0.048+-0.008
1500 	 10.217+-8.167 	 0.417+-0.155 	 0.892+-0.049 	 0.047+-0.008
2000 	 6.036+-6.266 	 0.280+-0.134 	 0.948+-0.041 	 0.048+-0.008
2500 	 5.770+-7.568 	 0.288+-0.232 	 0.954+-0.052 	 0.048+-0.008
3000 	 2.554+-7.050 	 0.181+-0.144 	 0.983+-0.032 	 0.048+-0.008
3500 	 2.356+-5.074 	 0.171+-0.126 	 0.985+-0.024 	 0.048+-0.008
4000 	 1.570+-2.408 	 0.134+-0.025 	 0.993+-0.009 	 0.043+-0.009
4500 	 5.215+-12.852 	 0.338+-0.661 	 0.937+-0.069 	 0.047+-0.008
5000 	 2.670+-4.745 	 0.170+-0.090 	 0.981+-0.027 	 0.046+-0.007
5500 	 0.803+-1.059 	 0.125+-0.028 	 0.994+-0.009 	

In [0]:
dataset = 'PROTEINS'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 32,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset PROTEINS. Samples 1113. Sample Features 61. Number of Classes 2.
Split 0. Train Samples 1001. Test Samples 112.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 484.962+-0.000 	 38.289+-0.000 	 0.562+-0.000 	 1.415+-0.000
500 	 51.883+-206.617 	 3.616+-24.606 	 0.660+-0.102 	 0.018+-0.002
1000 	 14.579+-10.892 	 0.919+-0.467 	 0.721+-0.092 	 0.018+-0.002
1500 	 9.254+-7.459 	 0.671+-0.260 	 0.768+-0.081 	 0.018+-0.002
2000 	 6.579+-4.488 	 0.527+-0.161 	 0.828+-0.078 	 0.018+-0.002
2500 	 5.562+-4.059 	 0.454+-0.146 	 0.860+-0.066 	 0.019+-0.002
3000 	 4.684+-5.234 	 0.359+-0.155 	 0.904+-0.066 	 0.020+-0.002
3500 	 4.967+-4.991 	 0.309+-0.134 	 0.928+-0.054 	 0.018+-0.002
4000 	 4.632+-5.194 	 0.314+-0.169 	 0.926+-0.063 	 0.018+-0.002
4500 	 4.601+-5.305 	 0.311+-0.200 	 0.929+-0.065 	 0.019+-0.002
5000 	 4.364+-4.851 	 0.278+-0.170 	 0.940+-0.058 	 0.019+-0.002
5500 	 1.846+-2.877 	 0.162+-0.067 	 0.984+-0.026 	 0.018+-0.002
6000 	 2.958+-5.129 	 0.194+-0.165 	

In [0]:
dataset = 'PROTEINS'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset PROTEINS. Samples 1113. Sample Features 61. Number of Classes 2.
Split 0. Train Samples 1001. Test Samples 112.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 1093.360+-0.000 	 46.348+-0.000 	 0.617+-0.000 	 1.542+-0.000
500 	 31.640+-105.191 	 1.929+-12.400 	 0.695+-0.098 	 0.024+-0.002
1000 	 12.082+-9.785 	 0.494+-0.188 	 0.858+-0.062 	 0.024+-0.002
1500 	 10.431+-30.652 	 0.524+-1.886 	 0.896+-0.063 	 0.024+-0.002
2000 	 6.814+-11.178 	 0.359+-0.250 	 0.935+-0.046 	 0.025+-0.002
2500 	 3.779+-5.021 	 0.262+-0.176 	 0.963+-0.046 	 0.024+-0.002
3000 	 1.361+-2.216 	 0.160+-0.031 	 0.992+-0.010 	 0.024+-0.002
3500 	 1.313+-3.263 	 0.155+-0.073 	 0.992+-0.016 	 0.024+-0.002
4000 	 1.192+-3.344 	 0.138+-0.065 	 0.993+-0.014 	 0.024+-0.002
4500 	 2.806+-6.470 	 0.206+-0.228 	 0.975+-0.047 	 0.024+-0.002
5000 	 2.330+-5.123 	 0.176+-0.170 	 0.979+-0.039 	 0.024+-0.002
5500 	 0.251+-0.356 	 0.107+-0.013 	 0.997+-0.005 	 0.023+-0.002
6000 	 1.572+-6.571 	 0.127+-0.15

In [0]:
dataset = 'PROTEINS'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 32,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset PROTEINS. Samples 1113. Sample Features 61. Number of Classes 2.
Split 0. Train Samples 1001. Test Samples 112.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 378.999+-0.000 	 34.481+-0.000 	 0.500+-0.000 	 1.644+-0.000
500 	 25.234+-27.825 	 1.811+-2.684 	 0.643+-0.094 	 0.024+-0.002
1000 	 7.010+-5.121 	 0.699+-0.212 	 0.718+-0.092 	 0.024+-0.002
1500 	 3.919+-2.446 	 0.592+-0.141 	 0.760+-0.085 	 0.025+-0.003
2000 	 3.359+-2.182 	 0.529+-0.122 	 0.795+-0.078 	 0.025+-0.002
2500 	 3.398+-2.370 	 0.505+-0.138 	 0.815+-0.078 	 0.024+-0.002
3000 	 3.537+-2.505 	 0.406+-0.116 	 0.867+-0.064 	 0.024+-0.002
3500 	 4.589+-4.004 	 0.395+-0.150 	 0.874+-0.071 	 0.024+-0.002
4000 	 4.371+-6.174 	 0.399+-0.187 	 0.873+-0.080 	 0.024+-0.002
4500 	 4.106+-3.533 	 0.327+-0.127 	 0.904+-0.060 	 0.024+-0.002
5000 	 4.442+-4.479 	 0.285+-0.134 	 0.924+-0.056 	 0.025+-0.002
5500 	 2.579+-3.019 	 0.162+-0.067 	 0.974+-0.031 	 0.024+-0.002
6000 	 4.100+-8.324 	 0.229+-0.250 	 0.9

In [0]:
dataset = 'PROTEINS'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 128,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Initializing folder at /root/grakel_data
Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset PROTEINS. Samples 1113. Sample Features 61. Number of Classes 2.
Split 0. Train Samples 1001. Test Samples 112.
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 95.501+-0.000 	 4.822+-0.000 	 0.484+-0.000 	 2.672+-0.000
500 	 16.702+-18.896 	 0.941+-1.521 	 0.702+-0.076 	 0.032+-0.003
1000 	 6.505+-4.761 	 0.481+-0.101 	 0.823+-0.050 	 0.031+-0.002
1500 	 4.816+-4.467 	 0.366+-0.135 	 0.881+-0.054 	 0.031+-0.003
2000 	 3.696+-2.738 	 0.308+-0.099 	 0.909+

In [0]:
dataset = 'PROTEINS'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 32,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Initializing folder at /root/grakel_data
Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset PROTEINS. Samples 1113. Sample Features 61. Number of Classes 2.
Split 0. Train Samples 1001. Test Samples 112.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 266.939+-0.000 	 27.252+-0.000 	 0.469+-0.000 	 2.226+-0.000
500 	 21.944+-22.222 	 1.747+-2.503 	 0.629+-0.102 	 0.023+-0.002
1000 	 5.268+-3.485 	 0.652+-0.148 	 0.712+-0.090 	 0.021+-0.002
1500 	 3.452+-2.486 	 0.582+-0.123 	 0.753+-0.080 	 0.021+-0.001
2000 	 3.209+-2.573 	 0.546+-0.143 	 0.775+-0.086 	 0.021+-0.001
2500 	 3.015+-2.115 	 0.

In [8]:
dataset = 'PROTEINS'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Initializing folder at /root/grakel_data
Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset PROTEINS. Samples 1113. Sample Features 61. Number of Classes 2.
Split 0. Train Samples 1001. Test Samples 112.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 222.385+-0.000 	 6.055+-0.000 	 0.430+-0.000 	 2.301+-0.000
500 	 11.721+-10.173 	 0.933+-1.714 	 0.700+-0.072 	 0.042+-0.002
1000 	 5.049+-3.679 	 0.468+-0.095 	 0.828+-0.049 	 0.042+-0.002
1500 	 4.368+-4.013 	 0.350+-0.118 	 0.887+-0.051 	 0.042+-0.002
2000 	 3.510+-2.717 	 0.259+-0.076 	 0.934+-0.031 	 0.042+-0.002
2500 	 3.023+-2.925 	 0.2

In [0]:
dataset = 'PROTEINS'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 32,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset PROTEINS. Samples 1113. Sample Features 61. Number of Classes 2.
Split 0. Train Samples 1001. Test Samples 112.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 367.986+-0.000 	 22.420+-0.000 	 0.438+-0.000 	 1.635+-0.000
500 	 23.681+-28.724 	 1.991+-5.226 	 0.646+-0.100 	 0.024+-0.002
1000 	 7.607+-4.678 	 0.659+-0.199 	 0.749+-0.088 	 0.025+-0.002
1500 	 4.744+-2.989 	 0.524+-0.169 	 0.810+-0.075 	 0.025+-0.002
2000 	 3.838+-2.910 	 0.437+-0.165 	 0.849+-0.071 	 0.024+-0.001
2500 	 3.512+-3.258 	 0.438+-0.181 	 0.858+-0.083 	 0.024+-0.002
3000 	 2.468+-2.381 	 0.320+-0.113 	 0.906+-0.060 	 0.024+-0.001
3500 	 2.525+-3.065 	 0.291+-0.152 	 0.921+-0.063 	 0.024+-0.002
4000 	 2.832+-3.077 	 0.281+-0.144 	 0.928+-0.055 	 0.026+-0.002
4500 	 2.578+-3.404 	 0.263+-0.169 	 0.934+-0.059 	 0.025+-0.002
5000 	 3.203+-3.725 	 0.261+-0.141 	 0.937+-0.049 	 0.024+-0.002
5500 	 1.129+-1.043 	 0.158+-0.058 	 0.975+-0.028 	 0.024+-0.001
6000 	 1.358+-2.991 	 0.138+-0.080 	 0.978+-0.028 	 0.024+-0.001
6500 	 1.822+-3.993 	 0.180+-0.149 	 0.968+-0.040 	 0.024+-0.002
7000 	 2.056+-7.550 	 0.161+-0.182 	 0.973+-0.037 	 0.02

In [0]:
dataset = 'PROTEINS'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 128,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Initializing folder at /root/grakel_data
Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset PROTEINS. Samples 1113. Sample Features 61. Number of Classes 2.
Split 0. Train Samples 1001. Test Samples 112.
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 223.206+-0.000 	 10.091+-0.000 	 0.523+-0.000 	 2.605+-0.000
500 	 14.792+-15.244 	 0.961+-2.246 	 0.729+-0.091 	 0.031+-0.002
1000 	 6.423+-5.301 	 0.367+-0.133 	 0.895+-0.054 	 0.030+-0.002
1500 	 3.872+-4.185 	 0.225+-0.100 	 0.960+-0.034 	 0.030+-0.002
2000 	 2.860+-3.831 	 0.230+-0.154 	 0.955+-0.052 	 0.030+-0.002
2500 	 2.816+-3.711 	 0.200+-0.118 	 0.967+-0.035 	 0.030+-0.002
3000 	 1.101+-2.013 	 0.117+-0.033 	 0.993+-0.010 	 0.030+-0.002
3500 	 1.818+-5.498 	 0.164+-0.217 	 0.979+-0.044 	 0.030+-0.002
4000 	 0.756+-1.251 	 0.097+-0.024 	 0.995+-0.009 	 0.030+-0.002
4500 	 1.549+-4.325 	 0.121+-0.113 	 0.987+-0.024 	 0.029+-0.002
5000 	 0.554+-1.058 	 0.081+-0.026 	 0.996+-0.010 	 0.030+-0.002
5500 	 0.150+-0.144 	 0.062+-0.005 	 0.998+-0.004 	 0.029+-0.002
6000 	 0.155+-0.219 	 0.056+-0.005 	 0.998+-0.004 	 0.029+-0.002
6500 	 2.068+-8.831 	 0.111+-0.276 	 0.988+-0.026 	 0.029+-0.002
7000 	 0.367+-0.460 	 0.057+-0.007 	 0.998+-0.004 	 0.02

In [0]:
dataset = 'PROTEINS'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 32,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset PROTEINS. Samples 1113. Sample Features 61. Number of Classes 2.
Split 0. Train Samples 1001. Test Samples 112.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 628.457+-0.000 	 32.534+-0.000 	 0.625+-0.000 	 1.701+-0.000
500 	 24.125+-24.838 	 1.675+-2.881 	 0.675+-0.102 	 0.025+-0.002
1000 	 7.079+-5.417 	 0.658+-0.239 	 0.764+-0.086 	 0.025+-0.002
1500 	 3.921+-2.715 	 0.497+-0.140 	 0.825+-0.072 	 0.025+-0.002
2000 	 3.509+-3.076 	 0.457+-0.179 	 0.848+-0.077 	 0.027+-0.003
2500 	 3.378+-2.596 	 0.419+-0.156 	 0.870+-0.070 	 0.025+-0.002
3000 	 2.552+-2.433 	 0.280+-0.105 	 0.933+-0.051 	 0.025+-0.002
3500 	 2.381+-2.135 	 0.226+-0.081 	 0.953+-0.039 	 0.025+-0.002
4000 	 3.110+-3.222 	 0.259+-0.143 	 0.942+-0.051 	 0.025+-0.002
4500 	 3.166+-4.049 	 0.268+-0.190 	 0.941+-0.052 	 0.026+-0.003
5000 	 2.245+-2.930 	 0.207+-0.122 	 0.961+-0.045 	 0.027+-0.003
5500 	 1.467+-2.302 	 0.167+-0.103 	 0.975+-0.039 	 0.025+-0.002
6000 	 1.238+-2.113 	 0.131+-0.061 	 0.986+-0.021 	 0.024+-0.002
6500 	 1.515+-3.632 	 0.137+-0.100 	 0.982+-0.028 	 0.024+-0.002
7000 	 1.955+-3.697 	 0.140+-0.093 	 0.979+-0.033 	 0.02

In [0]:
dataset = 'PROTEINS'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset PROTEINS. Samples 1113. Sample Features 61. Number of Classes 2.
Split 0. Train Samples 1001. Test Samples 112.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 279.008+-0.000 	 23.456+-0.000 	 0.352+-0.000 	 1.545+-0.000
500 	 13.669+-11.973 	 0.924+-1.839 	 0.744+-0.088 	 0.029+-0.003
1000 	 5.691+-4.984 	 0.366+-0.152 	 0.893+-0.055 	 0.029+-0.002
1500 	 3.644+-3.666 	 0.254+-0.121 	 0.941+-0.041 	 0.030+-0.004
2000 	 2.933+-3.871 	 0.237+-0.156 	 0.949+-0.053 	 0.045+-0.009
2500 	 2.342+-2.701 	 0.208+-0.103 	 0.958+-0.035 	 0.049+-0.007
3000 	 1.518+-2.659 	 0.127+-0.061 	 0.986+-0.018 	 0.049+-0.007
3500 	 1.394+-7.782 	 0.143+-0.336 	 0.982+-0.050 	 0.049+-0.007
4000 	 1.099+-1.258 	 0.123+-0.066 	 0.983+-0.027 	 0.049+-0.007
4500 	 1.131+-2.808 	 0.106+-0.082 	 0.987+-0.021 	 0.049+-0.007
5000 	 2.782+-9.036 	 0.245+-0.565 	 0.952+-0.076 	 0.049+-0.007
5500 	 1.300+-1.782 	 0.099+-0.025 	 0.989+-0.010 	 0.049+-0.007
6000 	 1.979+-3.715 	 0.092+-0.045 	 0.991+-0.012 	 0.049+-0.007
6500 	 1.782+-4.132 	 0.093+-0.078 	 0.992+-0.014 	 0.048+-0.007
7000 	 0.558+-1.109 	 0.063+-0.008 	 0.997+-0.005 	 0.04

In [0]:
dataset = 'NCI1'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 32,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset NCI1. Samples 4110. Sample Features 22. Number of Classes 2.
Split 0. Train Samples 3698. Test Samples 412.
Instructions for updating:
Use tf.cast instead.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 1392.996+-0.000 	 46.567+-0.000 	 0.531+-0.000 	 1.613+-0.000
500 	 31.200+-90.750 	 2.343+-16.021 	 0.575+-0.095 	 0.024+-0.002
1000 	 11.345+-5.282 	 0.851+-0.181 	 0.608+-0.088 	 0.024+-0.002
1500 	 7.174+-3.296 	 0.762+-0.138 	 0.627+-0.088 	 0.026+-0.002
2000 	 4.679+-2.055 	 0.699+-0.112 	 0.658+-0.091 	 0.024+-0.002
2500 	 3.698+-1.683 	 0.680+-0.101 	 0.664+-0.084 	 0.024+-0.002
3000 	 2.920+-1.357 	 0.645+-0.092 	 0.696+-0.080 	 0.024+-0.002
3500 	 2.911+-1.272 	 0.644+-0.091 	 0.690+-0.082 	 0.024+-0.002
4000 	 2.806+-1.233 	 0.634+-0.092 	 0.694+-0.086 	 0.024+-0.002
4500 	 2.688+-1.316 	 0.634+-0.101 	 0.698+-0.085 	 0.026+-0.002
5000 	 2.703+-1.282 	 0.630+-0.095 	 0.696+-0.084 	 0.024+-0.002
5500 	 2.428+-1.160 	 0.613+-0.093 	 0.714+-0.083 	 0.025+

In [0]:
dataset = 'NCI1'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 128,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset NCI1. Samples 4110. Sample Features 22. Number of Classes 2.
Split 0. Train Samples 3698. Test Samples 412.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 655.962+-0.000 	 31.613+-0.000 	 0.469+-0.000 	 1.710+-0.000
500 	 24.545+-54.744 	 1.480+-8.168 	 0.589+-0.058 	 0.027+-0.002
1000 	 9.448+-4.426 	 0.731+-0.080 	 0.656+-0.047 	 0.028+-0.003
1500 	 6.849+-3.560 	 0.682+-0.075 	 0.676+-0.047 	 0.028+-0.002
2000 	 4.320+-1.995 	 0.642+-0.056 	 0.696+-0.047 	 0.027+-0.002
2500 	 2.890+-1.388 	 0.613+-0.052 	 0.719+-0.042 	 0.027+-0.002
3000 	 2.383+-1.093 	 0.587+-0.047 	 0.740+-0.041 	 0.028+-0.003
3500 	 2.412+-1.034 	 0.576+-0.051 	 0.748+-0.041 	 0.027+-0.002
4000 	 2.486+-1.108 	 0.569+-0.049 	 0.752+-0.038 	 0.027+-0.002
4500 	 2.404+-1.122 	 0.559+-0.053 	 0.757+-0.040 	 0.027+-0.002
5000 	 2.330+-0.982 	 0.550+-0.052 	 0.765+-0.039 	 0.029+-0.003
5500 	 2.180+-0.861 	 0.519+-0.048 	 0.786+-0.036 	 0.027+-0.002
6000 	 2.416+-0.893 	 0.513+-0.051 	 0.788+-

In [0]:
dataset = 'NCI1'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 32,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset NCI1. Samples 4110. Sample Features 22. Number of Classes 2.
Split 0. Train Samples 3698. Test Samples 412.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 647.245+-0.000 	 24.076+-0.000 	 0.531+-0.000 	 1.560+-0.000
500 	 26.676+-37.891 	 1.608+-3.946 	 0.585+-0.089 	 0.021+-0.002
1000 	 8.388+-4.290 	 0.773+-0.150 	 0.627+-0.087 	 0.021+-0.002
1500 	 4.778+-2.240 	 0.698+-0.105 	 0.652+-0.086 	 0.021+-0.002
2000 	 3.306+-1.584 	 0.665+-0.095 	 0.670+-0.087 	 0.021+-0.002
2500 	 2.901+-1.419 	 0.651+-0.094 	 0.678+-0.082 	 0.021+-0.002
3000 	 2.474+-1.255 	 0.623+-0.089 	 0.697+-0.083 	 0.022+-0.002
3500 	 2.391+-1.327 	 0.623+-0.099 	 0.698+-0.089 	 0.022+-0.002
4000 	 2.528+-1.314 	 0.620+-0.090 	 0.697+-0.084 	 0.021+-0.002
4500 	 2.450+-1.353 	 0.612+-0.095 	 0.701+-0.083 	 0.021+-0.002
5000 	 2.663+-1.559 	 0.619+-0.098 	 0.697+-0.083 	 0.021+-0.002
5500 	 2.325+-1.237 	 0.596+-0.085 	 0.716+-0.080 	 0.021+-0.002
6000 	 2.337+-1.153 	 0.589+-0.088 	 0.720+-

In [0]:
dataset = 'NCI1'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset NCI1. Samples 4110. Sample Features 22. Number of Classes 2.
Split 0. Train Samples 3698. Test Samples 412.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 693.425+-0.000 	 27.689+-0.000 	 0.438+-0.000 	 1.909+-0.000
500 	 25.503+-40.122 	 1.528+-5.701 	 0.589+-0.060 	 0.024+-0.002
1000 	 11.191+-6.581 	 0.748+-0.113 	 0.647+-0.051 	 0.023+-0.002
1500 	 7.704+-4.024 	 0.687+-0.076 	 0.673+-0.047 	 0.022+-0.002
2000 	 5.057+-2.498 	 0.645+-0.056 	 0.696+-0.044 	 0.022+-0.002
2500 	 3.301+-1.760 	 0.619+-0.051 	 0.715+-0.043 	 0.024+-0.002
3000 	 2.574+-1.318 	 0.591+-0.050 	 0.739+-0.041 	 0.022+-0.002
3500 	 2.416+-1.179 	 0.583+-0.049 	 0.740+-0.041 	 0.023+-0.001
4000 	 2.301+-1.281 	 0.577+-0.049 	 0.744+-0.040 	 0.023+-0.002
4500 	 2.278+-1.153 	 0.569+-0.054 	 0.748+-0.042 	 0.023+-0.002
5000 	 2.336+-1.179 	 0.563+-0.050 	 0.751+-0.041 	 0.024+-0.002
5500 	 2.233+-1.017 	 0.539+-0.050 	 0.765+-0.038 	 0.022+-0.002
6000 	 2.511+-1.117 	 0.531+-0.050 	 0.771+

In [0]:
dataset = 'NCI1'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 32,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset NCI1. Samples 4110. Sample Features 22. Number of Classes 2.
Split 0. Train Samples 3698. Test Samples 412.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 197.824+-0.000 	 24.965+-0.000 	 0.531+-0.000 	 1.718+-0.000
500 	 22.271+-19.883 	 1.482+-1.996 	 0.567+-0.093 	 0.025+-0.002
1000 	 5.792+-2.894 	 0.761+-0.140 	 0.605+-0.085 	 0.025+-0.002
1500 	 2.818+-1.195 	 0.684+-0.088 	 0.629+-0.085 	 0.026+-0.002
2000 	 2.427+-1.260 	 0.662+-0.088 	 0.644+-0.087 	 0.025+-0.002
2500 	 2.411+-1.431 	 0.644+-0.090 	 0.660+-0.092 	 0.027+-0.003
3000 	 1.973+-1.012 	 0.619+-0.080 	 0.681+-0.080 	 0.026+-0.002
3500 	 2.029+-1.152 	 0.619+-0.084 	 0.679+-0.080 	 0.026+-0.002
4000 	 2.045+-1.050 	 0.614+-0.082 	 0.678+-0.085 	 0.026+-0.002
4500 	 2.017+-1.037 	 0.617+-0.089 	 0.677+-0.087 	 0.025+-0.002
5000 	 2.195+-1.121 	 0.618+-0.092 	 0.674+-0.084 	 0.026+-0.002
5500 	 1.867+-0.921 	 0.600+-0.077 	 0.690+-0.083 	 0.028+-0.002
6000 	 1.877+-0.912 	 0.598+-0.088 	 0.692+-

In [0]:
dataset = 'NCI1'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 128,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset NCI1. Samples 4110. Sample Features 22. Number of Classes 2.
Split 0. Train Samples 3698. Test Samples 412.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 153.196+-0.000 	 11.523+-0.000 	 0.523+-0.000 	 1.775+-0.000
500 	 19.008+-20.140 	 1.426+-4.929 	 0.565+-0.063 	 0.029+-0.002
1000 	 7.986+-4.629 	 0.728+-0.079 	 0.629+-0.049 	 0.029+-0.002
1500 	 4.724+-2.569 	 0.663+-0.057 	 0.672+-0.045 	 0.028+-0.002
2000 	 2.889+-1.659 	 0.635+-0.049 	 0.688+-0.045 	 0.030+-0.003
2500 	 1.961+-1.066 	 0.616+-0.046 	 0.700+-0.039 	 0.028+-0.003
3000 	 1.701+-0.955 	 0.599+-0.044 	 0.713+-0.040 	 0.028+-0.002
3500 	 1.853+-0.995 	 0.593+-0.044 	 0.716+-0.039 	 0.028+-0.003
4000 	 1.990+-1.130 	 0.589+-0.047 	 0.717+-0.042 	 0.031+-0.003
4500 	 1.806+-0.917 	 0.583+-0.045 	 0.719+-0.040 	 0.028+-0.002
5000 	 1.778+-0.922 	 0.581+-0.046 	 0.719+-0.042 	 0.028+-0.002
5500 	 1.669+-0.890 	 0.568+-0.046 	 0.729+-0.041 	 0.028+-0.002
6000 	 1.850+-0.962 	 0.566+-0.045 	 0.731+-

In [0]:
dataset = 'NCI1'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 32,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset NCI1. Samples 4110. Sample Features 22. Number of Classes 2.
Split 0. Train Samples 3698. Test Samples 412.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 165.020+-0.000 	 9.340+-0.000 	 0.500+-0.000 	 1.561+-0.000
500 	 21.089+-19.216 	 1.520+-2.631 	 0.547+-0.093 	 0.021+-0.002
1000 	 5.730+-2.779 	 0.766+-0.126 	 0.599+-0.086 	 0.021+-0.002
1500 	 3.165+-1.336 	 0.705+-0.085 	 0.616+-0.084 	 0.023+-0.002
2000 	 2.270+-1.105 	 0.674+-0.083 	 0.632+-0.085 	 0.023+-0.002
2500 	 2.222+-1.284 	 0.663+-0.081 	 0.637+-0.089 	 0.021+-0.002
3000 	 2.023+-1.026 	 0.633+-0.076 	 0.672+-0.085 	 0.021+-0.002
3500 	 1.969+-1.002 	 0.623+-0.084 	 0.680+-0.083 	 0.021+-0.002
4000 	 2.089+-1.085 	 0.619+-0.086 	 0.678+-0.085 	 0.021+-0.002
4500 	 2.151+-1.457 	 0.621+-0.093 	 0.680+-0.086 	 0.022+-0.002
5000 	 2.025+-1.113 	 0.612+-0.082 	 0.682+-0.083 	 0.021+-0.002
5500 	 1.884+-0.945 	 0.597+-0.083 	 0.697+-0.084 	 0.021+-0.002
6000 	 1.915+-0.977 	 0.603+-0.084 	 0.689+-0

In [0]:
dataset = 'NCI1'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Initializing folder at /root/grakel_data
Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset NCI1. Samples 4110. Sample Features 22. Number of Classes 2.
Split 0. Train Samples 3698. Test Samples 412.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 151.235+-0.000 	 10.950+-0.000 	 0.492+-0.000 	 2.986+-0.000
500 	 18.763+-21.043 	 1.231+-3.129 	 0.566+-0.056 	 0.038+-0.002
1000 	 6.319+-3.173 	 0.732+-0.069 	 0.618+-0.045 	 0.038+-0.002
1500 	 3.036+-1.759 	 0.678+-0.046 	 0.640+-0.044 	 0.038+-0.002
2000 	 2.111+-0.973 	 0.648+-0.041 	 0.661+-0.044 	 0.038+-0.002
2500 	 1.679+-0.833 	 0.627+

In [0]:
dataset = 'NCI1'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 32,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset NCI1. Samples 4110. Sample Features 22. Number of Classes 2.
Split 0. Train Samples 3698. Test Samples 412.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 229.960+-0.000 	 22.055+-0.000 	 0.500+-0.000 	 1.681+-0.000
500 	 21.066+-21.133 	 1.588+-3.046 	 0.562+-0.091 	 0.024+-0.002
1000 	 5.692+-2.777 	 0.765+-0.127 	 0.608+-0.086 	 0.025+-0.002
1500 	 3.081+-1.340 	 0.687+-0.089 	 0.643+-0.089 	 0.026+-0.003
2000 	 2.599+-1.190 	 0.667+-0.085 	 0.654+-0.078 	 0.025+-0.003
2500 	 2.375+-1.268 	 0.659+-0.090 	 0.656+-0.088 	 0.024+-0.002
3000 	 1.960+-1.012 	 0.636+-0.084 	 0.674+-0.081 	 0.024+-0.002
3500 	 1.837+-0.878 	 0.624+-0.085 	 0.684+-0.084 	 0.024+-0.002
4000 	 1.857+-0.892 	 0.621+-0.082 	 0.684+-0.082 	 0.026+-0.002
4500 	 2.000+-1.034 	 0.621+-0.083 	 0.679+-0.079 	 0.024+-0.002
5000 	 2.083+-1.047 	 0.623+-0.086 	 0.678+-0.083 	 0.024+-0.002
5500 	 1.744+-0.790 	 0.602+-0.082 	 0.697+-0.083 	 0.024+-0.002
6000 	 1.731+-0.861 	 0.603+-0.077 	 0.694+-0.079 	 0.025+-0.002
6500 	 1.783+-0.849 	 0.600+-0.081 	 0.696+-0.078 	 0.024+-0.002
7000 	 1.809+-0.780 	 0.599+-0.079 	 0.700+-0.076 	 0.02

In [0]:
dataset = 'NCI1'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 128,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset NCI1. Samples 4110. Sample Features 22. Number of Classes 2.
Split 0. Train Samples 3698. Test Samples 412.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 302.461+-0.000 	 45.084+-0.000 	 0.453+-0.000 	 1.871+-0.000
500 	 17.905+-18.594 	 1.150+-2.488 	 0.593+-0.058 	 0.041+-0.002
1000 	 6.841+-3.346 	 0.713+-0.077 	 0.651+-0.046 	 0.041+-0.002
1500 	 3.509+-1.525 	 0.651+-0.052 	 0.680+-0.043 	 0.042+-0.002
2000 	 2.248+-0.941 	 0.624+-0.045 	 0.697+-0.040 	 0.042+-0.002
2500 	 1.698+-0.801 	 0.611+-0.044 	 0.705+-0.041 	 0.041+-0.002
3000 	 1.309+-0.556 	 0.590+-0.044 	 0.723+-0.040 	 0.042+-0.002
3500 	 1.418+-0.614 	 0.589+-0.041 	 0.722+-0.040 	 0.042+-0.002
4000 	 1.496+-0.838 	 0.587+-0.046 	 0.722+-0.041 	 0.042+-0.002
4500 	 1.518+-0.818 	 0.580+-0.046 	 0.726+-0.042 	 0.042+-0.002
5000 	 1.754+-0.937 	 0.573+-0.047 	 0.729+-0.040 	 0.041+-0.002
5500 	 1.437+-0.659 	 0.551+-0.046 	 0.749+-0.041 	 0.041+-0.002
6000 	 1.511+-0.749 	 0.548+-0.046 	 0.750+-0.038 	 0.041+-0.002
6500 	 1.580+-0.766 	 0.549+-0.045 	 0.749+-0.037 	 0.041+-0.002
7000 	 1.534+-0.754 	 0.544+-0.043 	 0.751+-0.036 	 0.04

In [0]:
dataset = 'NCI1'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 32,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Initializing folder at /root/grakel_data
Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset NCI1. Samples 4110. Sample Features 22. Number of Classes 2.
Split 0. Train Samples 3698. Test Samples 412.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 227.715+-0.000 	 14.472+-0.000 	 0.500+-0.000 	 2.105+-0.000
500 	 21.253+-22.584 	 1.552+-4.799 	 0.569+-0.089 	 0.020+-0.001
1000 	 6.337+-3.093 	 0.758+-0.129 	 0.619+-0.089 	 0.020+-0.001
1500 	 3.435+-1.634 	 0.674+-0.104 	 0.651+-0.091 	 0.022+-0.002
2000 	 2.514+-1.136 	 0.644+-0.093 	 0.677+-0.087 	 0.020+-0.001
2500 	 2.475+-1.518 	 0.642+-0.092 	 0.671+-0.079 	 0.020+-0.001
3000 	 2.156+-0.966 	 0.643+-0.078 	 0.662+-0.086 	 0.020+-0.001
3500 	 2.134+-1.109 	 0.627+-0.085 	 0.674+-0.090 	 0.020+-0.001
4000 	 2.190+-1.122 	 0.623+-0.082 	 0.680+-0.082 	 0.020+-0.001
4500 	 2.171+-1.012 	 0.617+-0.083 	 0.682+-0.080 	 0.020+-0.001
5000 	 2.176+-1.017 	 0.620+-0.085 	 0.677+-0.081 	 0.022+-0.002
5500 	 1.914+-0.845 	 0.601+-0.082 	 0.699+-0.079 	 0.023+-0.002
6000 	 1.983+-0.965 	 0.597+-0.085 	 0.698+-0.082 	 0.020+-0.001
6500 	 2.044+-0.970 	 0.597+-0.082 	 0.696+-0.082 	 0.020+-0.001
7000 	 2.178+-1.075 	 0.599+-0.087 	 0.695+-0.080 	 0.02

In [0]:
dataset = 'NCI1'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset NCI1. Samples 4110. Sample Features 22. Number of Classes 2.
Split 0. Train Samples 3698. Test Samples 412.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 107.364+-0.000 	 17.701+-0.000 	 0.508+-0.000 	 1.765+-0.000
500 	 20.468+-18.497 	 1.165+-1.884 	 0.605+-0.054 	 0.040+-0.002
1000 	 7.669+-3.797 	 0.719+-0.082 	 0.648+-0.049 	 0.040+-0.002
1500 	 3.965+-1.838 	 0.651+-0.055 	 0.673+-0.045 	 0.040+-0.002
2000 	 2.475+-1.004 	 0.628+-0.043 	 0.682+-0.041 	 0.040+-0.002
2500 	 1.788+-0.854 	 0.616+-0.043 	 0.691+-0.041 	 0.040+-0.002
3000 	 1.485+-0.701 	 0.595+-0.041 	 0.708+-0.039 	 0.040+-0.002
3500 	 1.569+-0.803 	 0.593+-0.042 	 0.709+-0.042 	 0.040+-0.002
4000 	 1.535+-0.772 	 0.586+-0.042 	 0.712+-0.039 	 0.040+-0.002
4500 	 1.635+-0.881 	 0.580+-0.044 	 0.715+-0.042 	 0.040+-0.002
5000 	 1.671+-0.888 	 0.577+-0.045 	 0.716+-0.042 	 0.040+-0.002
5500 	 1.457+-0.710 	 0.560+-0.045 	 0.730+-0.040 	 0.040+-0.002
6000 	 1.665+-0.874 	 0.556+-0.043 	 0.730+-0.038 	 0.040+-0.002
6500 	 1.661+-0.841 	 0.551+-0.044 	 0.737+-0.037 	 0.040+-0.002
7000 	 1.658+-0.814 	 0.549+-0.045 	 0.740+-0.039 	 0.04

In [0]:
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 32,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Initializing folder at /root/grakel_data
Extracting dataset  IMDB-BINARY..
Parsing dataset  IMDB-BINARY..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset IMDB-BINARY. Samples 1000. Sample Features 65. Number of Classes 2.
Split 0. Train Samples 900. Test Samples 100.
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 37999.770+-0.000 	 1704.608+-0.000 	 0.594+-0.000 	 2.296+-0.000
500 	 8537.598+-60294.520 	 321.475+-2806.871 	 0.629+-0.101 	 0.024+-0.002
1000 	 46.561+-25.891 	 1.476+-0.610 	 0.648+-0.094 	 0.023+-0.002
1500 	 31.044+-17.190 	 1.204+-0.489 	 0.669+-0.095 	 0.023+-0.002
2000 	 26.589+-16.081

In [0]:
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 128,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  IMDB-BINARY..
Parsing dataset  IMDB-BINARY..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset IMDB-BINARY. Samples 1000. Sample Features 65. Number of Classes 2.
Split 0. Train Samples 900. Test Samples 100.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 136754.922+-0.000 	 4422.382+-0.000 	 0.531+-0.000 	 1.421+-0.000
500 	 2207.885+-20387.299 	 92.300+-977.870 	 0.666+-0.107 	 0.027+-0.002
1000 	 21.298+-20.665 	 0.909+-0.482 	 0.722+-0.096 	 0.028+-0.003
1500 	 19.425+-14.368 	 0.829+-0.296 	 0.735+-0.091 	 0.029+-0.003
2000 	 17.135+-27.233 	 0.774+-0.305 	 0.752+-0.092 	 0.027+-0.002
2500 	 11022.220+-90757.039 	 634.895+-5493.180 	 0.675+-0.132 	 0.028+-0.003
3000 	 98.117+-73.371 	 2.821+-1.528 	 0.649+-0.103 	 0.027+-0.002
3500 	 86.209+-58.977 	 2.141+-1.119 	 0.674+-0.095 	 0.029+-0.003
4000 	 84.073+-64.584 	 1.999+-0.857 	 0.678+-0.091 	 0.027+-0.002
4500 	 77.561+-60.607 	 1.883+-0.864 	 0.674+-0.100 	 0.027+-0.002
5000 	 74.017+-50.727 	 1.798+-0.747 	 0.693+-0.092 	 0.027+-0.002
5500 	 61.600+-46.261 	 1.455+-0.421 	 0.690+-0.087 	 0.029+-

In [0]:
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 32,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  IMDB-BINARY..
Parsing dataset  IMDB-BINARY..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset IMDB-BINARY. Samples 1000. Sample Features 65. Number of Classes 2.
Split 0. Train Samples 900. Test Samples 100.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 77527.758+-0.000 	 7203.372+-0.000 	 0.500+-0.000 	 1.709+-0.000
500 	 3436.033+-26427.328 	 215.245+-2834.800 	 0.673+-0.103 	 0.019+-0.002
1000 	 38.695+-29.550 	 1.097+-0.481 	 0.706+-0.095 	 0.018+-0.002
1500 	 25.694+-18.890 	 0.840+-0.282 	 0.722+-0.095 	 0.018+-0.002
2000 	 25.998+-33.097 	 0.873+-0.378 	 0.733+-0.089 	 0.018+-0.002
2500 	 14573.526+-111106.805 	 898.132+-8396.185 	 0.610+-0.116 	 0.019+-0.002
3000 	 101.763+-79.295 	 2.170+-1.086 	 0.657+-0.090 	 0.021+-0.002
3500 	 107.882+-61.238 	 2.045+-0.840 	 0.651+-0.097 	 0.020+-0.002
4000 	 92.058+-47.852 	 1.758+-0.531 	 0.663+-0.098 	 0.020+-0.002
4500 	 89.748+-61.101 	 1.950+-1.006 	 0.662+-0.101 	 0.019+-0.002
5000 	 77.351+-43.927 	 1.676+-0.582 	 0.672+-0.091 	 0.019+-0.002
5500 	 66.354+-40.298 	 1.402+-0.418 	 0.683+-0.099 	 0.0

In [0]:
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'sum',
    batch_size = 128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Initializing folder at /root/grakel_data
Extracting dataset  IMDB-BINARY..
Parsing dataset  IMDB-BINARY..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset IMDB-BINARY. Samples 1000. Sample Features 65. Number of Classes 2.
Split 0. Train Samples 900. Test Samples 100.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 224384.109+-0.000 	 1512.437+-0.000 	 0.547+-0.000 	 2.112+-0.000
500 	 1298.714+-10601.064 	 71.815+-823.048 	 0.658+-0.095 	 0.019+-0.001
1000 	 25.753+-29.354 	 1.082+-0.798 	 0.685+-0.096 	 0.020+-0.002
1500 	 15.693+-24.445 	 0.782+-0.323 	 0.728+-0.095 	 0.021+-0.002
2000 	 11.105+-9.046 	 0.687+-0.225 	 0.757+-0.084 	 0.020+-0.002
2500

In [0]:
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 32,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Initializing folder at /root/grakel_data
Extracting dataset  IMDB-BINARY..
Parsing dataset  IMDB-BINARY..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset IMDB-BINARY. Samples 1000. Sample Features 65. Number of Classes 2.
Split 0. Train Samples 900. Test Samples 100.
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 158.199+-0.000 	 22.622+-0.000 	 0.375+-0.000 	 2.441+-0.000
500 	 12.852+-13.491 	 1.156+-1.340 	 0.667+-0.101 	 0.027+-0.002
1000 	 3.738+-1.886 	 0.565+-0.130 	 0.746+-0.088 	 0.027+-0.002
1500 	 2.554+-1.295 	 0.486+-0.114 	 0.780+-0.084 	 0.026+-0.002
2000 	 2.867+-2.467 	 0.492+-0.138 	 0.

In [0]:
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 128,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  IMDB-BINARY..
Parsing dataset  IMDB-BINARY..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset IMDB-BINARY. Samples 1000. Sample Features 65. Number of Classes 2.
Split 0. Train Samples 900. Test Samples 100.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 125.804+-0.000 	 8.321+-0.000 	 0.500+-0.000 	 1.597+-0.000
500 	 10.461+-12.586 	 1.056+-2.591 	 0.690+-0.106 	 0.033+-0.004
1000 	 3.316+-2.923 	 0.610+-0.186 	 0.725+-0.093 	 0.030+-0.003
1500 	 2.472+-2.207 	 0.526+-0.114 	 0.757+-0.079 	 0.030+-0.003
2000 	 2.171+-2.093 	 0.508+-0.129 	 0.763+-0.088 	 0.030+-0.003
2500 	 2.045+-2.127 	 0.438+-0.106 	 0.801+-0.076 	 0.031+-0.004
3000 	 2.083+-3.369 	 0.403+-0.130 	 0.822+-0.088 	 0.030+-0.003
3500 	 1.745+-1.876 	 0.400+-0.111 	 0.816+-0.075 	 0.030+-0.003
4000 	 1.626+-1.834 	 0.343+-0.088 	 0.838+-0.076 	 0.030+-0.003
4500 	 1.717+-1.718 	 0.339+-0.084 	 0.842+-0.069 	 0.032+-0.004
5000 	 1.769+-2.085 	 0.327+-0.085 	 0.847+-0.063 	 0.031+-0.003
5500 	 1.396+-1.547 	 0.314+-0.083 	 0.851+-0.075 	 0.030+-0.003
6000 	 1.793+-5.047 	 0.301+-0.078 	 0.

In [0]:
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 32,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  IMDB-BINARY..
Parsing dataset  IMDB-BINARY..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset IMDB-BINARY. Samples 1000. Sample Features 65. Number of Classes 2.
Split 0. Train Samples 900. Test Samples 100.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 108.984+-0.000 	 14.863+-0.000 	 0.500+-0.000 	 1.729+-0.000
500 	 13.529+-17.577 	 1.187+-3.261 	 0.671+-0.102 	 0.023+-0.002
1000 	 4.227+-2.358 	 0.597+-0.147 	 0.738+-0.085 	 0.021+-0.002
1500 	 2.880+-1.614 	 0.534+-0.128 	 0.757+-0.090 	 0.021+-0.001
2000 	 2.361+-1.708 	 0.500+-0.129 	 0.775+-0.087 	 0.021+-0.002
2500 	 2.091+-1.203 	 0.467+-0.105 	 0.788+-0.077 	 0.021+-0.002
3000 	 1.852+-1.398 	 0.396+-0.093 	 0.828+-0.076 	 0.021+-0.001
3500 	 2.025+-2.228 	 0.384+-0.114 	 0.831+-0.078 	 0.023+-0.002
4000 	 2.145+-1.745 	 0.379+-0.116 	 0.832+-0.079 	 0.021+-0.001
4500 	 2.103+-1.548 	 0.357+-0.093 	 0.837+-0.067 	 0.021+-0.002
5000 	 2.257+-2.522 	 0.350+-0.115 	 0.841+-0.070 	 0.021+-0.001
5500 	 2.110+-2.693 	 0.336+-0.108 	 0.850+-0.070 	 0.021+-0.002
6000 	 1.831+-1.468 	 0.299+-0.083 	 0

KeyboardInterrupt: ignored

In [0]:
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'mean',
    batch_size = 128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Initializing folder at /root/grakel_data
Extracting dataset  IMDB-BINARY..
Parsing dataset  IMDB-BINARY..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset IMDB-BINARY. Samples 1000. Sample Features 65. Number of Classes 2.
Split 0. Train Samples 900. Test Samples 100.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 109.095+-0.000 	 8.987+-0.000 	 0.531+-0.000 	 2.339+-0.000
500 	 11.228+-13.770 	 1.067+-2.546 	 0.691+-0.096 	 0.022+-0.002
1000 	 3.378+-3.375 	 0.569+-0.145 	 0.741+-0.095 	 0.022+-0.002
1500 	 2.139+-1.833 	 0.486+-0.103 	 0.778+-0.076 	 0.022+-0.002
2000 	 2.032+-1.960 	 0.471+-0.113 	 0.784+-0.079 	 0.023+-0.002
2500 	 1.726+-1.524 	 0

In [0]:
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 32,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

In [0]:
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 128,      
    with_eps = True,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  IMDB-BINARY..
Parsing dataset  IMDB-BINARY..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset IMDB-BINARY. Samples 1000. Sample Features 65. Number of Classes 2.
Split 0. Train Samples 900. Test Samples 100.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 110.503+-0.000 	 11.333+-0.000 	 0.539+-0.000 	 1.512+-0.000
500 	 10.152+-12.456 	 0.982+-2.499 	 0.706+-0.097 	 0.029+-0.003
1000 	 2.634+-1.862 	 0.467+-0.093 	 0.794+-0.072 	 0.030+-0.004
1500 	 2.321+-2.477 	 0.465+-0.132 	 0.790+-0.076 	 0.029+-0.003
2000 	 1.605+-1.557 	 0.421+-0.106 	 0.810+-0.072 	 0.029+-0.003
2500 	 1.544+-2.009 	 0.402+-0.153 	 0.823+-0.075 	 0.029+-0.003
3000 	 1.199+-1.222 	 0.339+-0.090 	 0.849+-0.071 	 0.030+-0.003
3500 	 1.499+-2.559 	 0.332+-0.112 	 0.848+-0.069 	 0.029+-0.003
4000 	 1.438+-1.673 	 0.341+-0.102 	 0.834+-0.086 	 0.031+-0.004
4500 	 1.237+-1.534 	 0.327+-0.104 	 0.838+-0.079 	 0.029+-0.003
5000 	 1.133+-1.444 	 0.314+-0.083 	 0.846+-0.069 	 0.030+-0.003
5500 	 0.964+-1.269 	 0.282+-0.074 	 0.863+-0.072 	 0.030+-0.003
6000 	 0.854+-0.969 	 0.275+-0.082 	 0.864+-0.062 	 0.029+-0.003
6500 	 1.077+-1.942 	 0.273+-0.102 	 0.864+-0.063 	 0.029+-0.003
7000 	 1.091+-1.270 	 0.269+-0.075 	 0.870+-0.067 	 0.03

In [0]:
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 32,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

In [0]:
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  IMDB-BINARY..
Parsing dataset  IMDB-BINARY..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset IMDB-BINARY. Samples 1000. Sample Features 65. Number of Classes 2.
Split 0. Train Samples 900. Test Samples 100.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 172.245+-0.000 	 22.244+-0.000 	 0.531+-0.000 	 1.617+-0.000
500 	 8.922+-10.995 	 0.865+-1.477 	 0.728+-0.098 	 0.030+-0.003
1000 	 3.214+-3.544 	 0.521+-0.188 	 0.785+-0.090 	 0.031+-0.003
1500 	 1.699+-1.466 	 0.419+-0.112 	 0.824+-0.069 	 0.030+-0.003
2000 	 1.699+-2.137 	 0.392+-0.143 	 0.828+-0.086 	 0.030+-0.003
2500 	 1.792+-2.246 	 0.414+-0.142 	 0.820+-0.083 	 0.030+-0.003
3000 	 1.741+-4.302 	 0.375+-0.147 	 0.838+-0.072 	 0.031+-0.004
3500 	 1.170+-1.181 	 0.337+-0.095 	 0.852+-0.074 	 0.030+-0.003
4000 	 1.365+-1.848 	 0.319+-0.092 	 0.856+-0.069 	 0.029+-0.003
4500 	 1.157+-1.201 	 0.301+-0.082 	 0.860+-0.074 	 0.030+-0.004
5000 	 1.196+-1.425 	 0.292+-0.090 	 0.865+-0.071 	 0.032+-0.004
5500 	 0.930+-0.864 	 0.268+-0.071 	 0.876+-0.069 	 0.030+-0.004
6000 	 0.911+-0.932 	 0.265+-0.071 	 0.871+-0.064 	 0.030+-0.003
6500 	 1.251+-2.536 	 0.267+-0.078 	 0.877+-0.056 	 0.030+-0.003
7000 	 1.194+-2.681 	 0.287+-0.270 	 0.869+-0.064 	 0.031

In [0]:
# fixed bias
dataset = 'IMDB-BINARY'
run(dataset,
    neighbor_pooling = 'max',
    batch_size = 128,      
    with_eps = False,
    ohe_input = True,
    save_files = True,
    folder = folder)

Extracting dataset  IMDB-BINARY..
Parsing dataset  IMDB-BINARY..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/grakel/graph.py:305: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


Finished extracting data.
Dataset IMDB-BINARY. Samples 1000. Sample Features 65. Number of Classes 2.
Split 0. Train Samples 900. Test Samples 100.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


IT 	 NORM 		 LOSS 		 ACC 		 TIME
0 	 79.222+-0.000 	 7.703+-0.000 	 0.516+-0.000 	 1.638+-0.000
500 	 8.240+-11.441 	 0.866+-2.073 	 0.719+-0.099 	 0.040+-0.005
1000 	 2.462+-2.157 	 0.465+-0.098 	 0.800+-0.075 	 0.040+-0.005
1500 	 2.019+-2.081 	 0.443+-0.112 	 0.799+-0.084 	 0.040+-0.005
2000 	 1.748+-2.042 	 0.408+-0.121 	 0.814+-0.078 	 0.040+-0.005
2500 	 1.460+-1.471 	 0.405+-0.125 	 0.807+-0.085 	 0.040+-0.005
3000 	 1.111+-1.142 	 0.314+-0.076 	 0.852+-0.065 	 0.041+-0.005
3500 	 1.166+-1.323 	 0.302+-0.082 	 0.859+-0.067 	 0.040+-0.005
4000 	 1.361+-2.418 	 0.316+-0.120 	 0.857+-0.073 	 0.040+-0.005
4500 	 1.157+-1.305 	 0.291+-0.090 	 0.861+-0.075 	 0.040+-0.005
5000 	 1.419+-2.328 	 0.320+-0.105 	 0.848+-0.070 	 0.041+-0.005
5500 	 0.989+-1.150 	 0.276+-0.085 	 0.869+-0.068 	 0.040+-0.005
6000 	 0.981+-1.644 	 0.271+-0.081 	 0.869+-0.068 	 0.040+-0.005
6500 	 0.939+-1.307 	 0.264+-0.075 	 0.872+-0.060 	 0.040+-0.005
7000 	 1.003+-1.640 	 0.261+-0.080 	 0.878+-0.058 	 0.040+-